<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/CSC645/blob/master/mnist-multilayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A deep feedforward network with arbitrary depth

In this exercise we build a deep feedforward neural network to recognize handwrittne digits (the MNIST data). The code has many tweekable parameters, in particular the number and width of layers. But as you will see adding more layers slows down the convergence considerably.


The usual imports plus gzip and pickle to read the data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import gzip
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Important to be able to diplay matplotlib plots inline in the notebook

In [ ]:
%matplotlib inline

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

The data are in compressed pickle format. 

In [ ]:
def load_dataset():
    #with gzip.open("/content/drive/My Drive/mnist.pkl.gz") as f:
    #    data=pickle._Unpickler(f)
    #    data.encoding='latin1'
    #    tr,_,te=data.load()
    tr,te=keras.datasets.mnist.load_data()
    print(tr[0].shape)
    print(te[0].shape)
    X=tr[0].reshape(60000,784)
    Y=tr[1]
    X=X.T
    Y=Y.reshape(1,len(Y))
    V=np.zeros((10,Y.shape[1]))
    for j in range(Y.shape[1]):
        V[Y[0,j],j]=1
        
    Y=V
    test_data=te[0].reshape(10000,784)
    test_labels=te[1]
    test_data=test_data.T
    test_labels=test_labels.reshape(1,len(test_labels))
    return X.astype("float32")/255,Y.astype("float32"),test_data.astype("float32")/255,test_labels.astype("float32")


The activation function is sigmoid

In [ ]:
def sigmoid(x):
    s = 1/(1+np.exp(-x))
    return s

This function plots some of the misclassified data in order for us to have an idea what when wrong.

In [ ]:
def print_misclassified(test_data,test_labels,biases,weights):
    As=forward_propagation(test_data,biases,weights)
    output=As[-1]
    count=0
    fig=plt.figure()
    fig.tight_layout()
    plt.subplots_adjust( wspace=1, hspace=1)

    for i in range(output.shape[1]):
        label=np.argmax(output[:,i])
        if label != test_labels[0,i]:
            if count>40:
                break
            subfig=count%40+1
            count=count+1
            img=test_data[:,i].reshape(28,28)
            t=fig.add_subplot(4,10,subfig)
            #t.set_title(str(i))
            t.set_title(str(label))
            t.axes.get_xaxis().set_visible(False)
            t.axes.get_yaxis().set_visible(False)
            plt.imshow(img,cmap='gray_r')



Returns the number of correct predictions

In [ ]:
def evaluate(test_data,biases,weights):
    As=forward_propagation(test_data,biases,weights)
    output=As[-1]
    count=0
    for i in range(output.shape[1]):
        if np.argmax(output[:,i])==test_labels[0,i]:
            count=count+1
    return count

The usual cross Entropy cost

In [ ]:
def compute_cost(Y,b,w):
    
    m = Y.shape[1] # number of example

    # Compute the cross-entropy cost
    As=forward_propagation(X,b,w)
    # recall that As contains the "output" of all layers
    # including the input As[0] and the final output As[-1]
    output=As[-1]
    logprobs = np.log(output)*Y+np.log(1-output)*(1-Y)
    cost = -np.sum(logprobs)/m

    count=0
    for i in range(output.shape[1]):
        if (np.argmax(output[:,i])==np.argmax(Y[:,i])):
            count=count+1

    
    cost = np.squeeze(cost)     # makes sure cost is the dimension we expect. 
                                # E.g., turns [[17]] into 17 
    assert(isinstance(cost, float))
    
    return cost,count


The weights are initialized randomly and the biases are initially set to zero

In [ ]:
def initialize_parameters(width):    
    weights=[]
    biases=[]
    for i in range(len(width)-1):
        print("size {},{}".format(width[i+1],width[i]))
        w=np.random.randn(width[i+1],width[i])
        b=np.zeros((width[i+1],1))
        weights.append(w)
        biases.append(b)

    return biases,weights


Forward propagation over all the layers but also retain the intermediate results.
For example below As[0] is the input, As[1] is the output of the first layer,..., and As[-1] (the last) is the output

In [ ]:
def forward_propagation(X,biases,weights):
    a=X
    As=[X]
    for w,b in zip(weights,biases):
       z=np.dot(w,a)+b
       a=sigmoid(z)
       As.append(a)

    return As

In [ ]:
def backward_propagation(X,Y,biases,weights):

    As=forward_propagation(X,biases,weights)
    m = X.shape[1]
    
    nlayers=len(biases)
    dz=As[nlayers]-Y
#   uncomment below if you want to use mean squared error
#    dz=dz*As[nlayers]*(1-As[nlayers])
    gradb=[]
    gradw=[]
    for i in range(nlayers,0,-1):
        db=np.sum(dz,axis=1,keepdims=True)/m
        dw=np.dot(dz,As[i-1].T)/m
        dz=np.dot(weights[i-1].T,dz)*As[i-1]*(1-As[i-1])
        gradb.insert(0,db)
        gradw.insert(0,dw)
    
    return gradb,gradw

In [ ]:
def update_parameters(biases,weights, gradb,gradw, learning_rate):


    for i in range(len(biases)):
        weights[i]=weights[i]-learning_rate*gradw[i]
        biases[i]=biases[i]-learning_rate*gradb[i]

    return biases,weights

Initialize the parameters


In [ ]:
def GD(X, Y, test_data,width,batch_size,num_iterations, learning_rate,print_cost=False):

    #n_x=X.shape[0]
    #n_y=Y.shape[0]
    biases,weights=initialize_parameters(width)
    
    for i in range(0, num_iterations):
        cost,count = compute_cost(Y,biases,weights)
        for k in range(0,X.shape[1],batch_size):
            idx=[random.randint(0,Y.shape[1]-1) for s in range(batch_size)]
            yb=Y[:,idx]
            xb=X[:,idx]
            gradb,gradw=backward_propagation(xb,yb,biases,weights)
            biases,weights=update_parameters(biases,weights,gradb,gradw,learning_rate)    

        if i%1 ==0 : 
            print(len(weights))   
            count_test=evaluate(test_data,biases,weights)
            print ("Cost after iteration %i: %f" %(i, cost))
            print("count after iteration %i is %i" %(i,count))
            print("test count after iteration %i is %i" %(i,count_test))
    return biases,weights

In [ ]:
X, Y,test_data,test_labels = load_dataset()
n_x=X.shape[0]
n_y=Y.shape[0]
width=[n_x,128,64,n_y]
biases,weights= GD(X, Y,test_data,width,batch_size=64,learning_rate=3,
                num_iterations =10, print_cost=True)


count=evaluate(test_data,biases,weights)
A=forward_propagation(X,biases,weights)
print(A[-1].shape)
print(len(A))


Prints some of the misclassified digits. The top on each digit shows the (wrong) prediction.

In [ ]:
print_misclassified(test_data,test_labels,biases,weights)
